In [1]:
import pandas as pd
import pandas_profiling
import numpy as np

In [2]:
%%javascript
require.config({
paths: {
d3: "https://d3js.org/d3.v5.min",
}
});

require(["d3"], function(d3) {
window.d3 = d3;
});

<IPython.core.display.Javascript object>

In [ ]:
columns = ["symboling", "normalized-losses", "make", "fuel-type", "aspiration", "num-of-doors", "body-style", "drive-wheels", "engine-location", "wheel-base", "length", "width", "height",
           "curb-weight", "engine-type", "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower", "peak-rpm", "city-mpg", "highway-mpg", "price"]

df = pd.read_csv('imports-85.data', names=columns)

for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')

df.replace(to_replace=["na", "?", np.nan, "missing", "not available","n/a", "missing value"], value=np.nan, inplace=True)

pandas_profiling.ProfileReport(df)

In [3]:
columns = ["class", "age", "menopause", "tumor-size", "inv-nodes", 
           "node-caps", "deg-malig", "breast", "breast-quad", "irradiat"]

df = pd.read_csv('breast-cancer.data', names=columns)

df['deg-malig'].replace(to_replace={1: 'one', 2:'two', 3:'three'}, inplace=True)

df.replace(to_replace=["na", "?", np.nan, "missing", "not available","n/a", "missing value"], value=np.nan, inplace=True)


pandas_profiling.ProfileReport(df)

c:\users\lucasbertogliociocar\appdata\local\programs\python\python37\lib\site-packages\pandas_profiling\describe.py:576: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)
